In [1]:
import pandas as pd
import numpy as np
import time
from ortools.linear_solver import pywraplp
import scipy
import matplotlib.pyplot as plt
#matplotlib.use('TkAgg')
from inspect import currentframe, getframeinfo
from pathlib import Path


filename = getframeinfo(currentframe()).filename
parent = Path(filename).resolve().parent
print (parent)
path = parent

# Загружаем ваш файл в переменную `file` / вместо 'example' укажите название свого файла из текущей директории
file = ('data/Исходная_выгрузка_ТМС_python_Excel.xlsx')
# Загружаем spreadsheet в объект pandas
xl = pd.ExcelFile(file)
df = xl.parse('Исходная выгрузка_ТМС_python')

df.columns

calulation_columns = ['Расчетный номер машины (из ТМС, 1С УАТ)', 'Наличие автомашины в собственном парке Почты', \
                      'Сумма весов емкостей из Сортмастер, кг', 'Средняя скорость передвижения', \
'Дата начала перевозки', 'Дата начала перевозки локальное время', 'Дата окончания перевозки (локальное время)','Фактическая грузоподъемность ТС (в кг)', 'УФПС начала перевозки', \
'Наименование маршрута', 'Модель ТС', 'Фактическое расстояние', 'Стоимость расходной части перевозки', \
'Расчет затрат (руб) на (тонна*км) по перевозкам и из ТМС, и из 1с уат']
df.dtypes

df['УФПС начала перевозки'].unique() #['УФПС МОСКОВСКОЙ ОБЛ', 'УФПС МОСКВЫ']

df = df[calulation_columns]
df['затраты на км'] = df['Стоимость расходной части перевозки']/df['Фактическое расстояние']

#оставим в выборке только те маршруты, которые покрываются за 24 часа, т.е. за 1 день.
df['Перевозка одним днем'] = np.where(
    df['Дата начала перевозки локальное время'].dt.day == df['Дата окончания перевозки (локальное время)'].dt.day, 1, 0)


#оценим, что мы вообще оптимизируем, какой процент в костах дадут оптимизируемые маршруты.
#1.маршруты 1 дня, сколько % от общего
#df[df['Перевозка одним днем']==1]['Стоимость расходной части перевозки'].sum()/df['Стоимость расходной части перевозки'].sum() #0.245 ?!!!!
#df[df['Дата начала перевозки локальное время'].isnull()]['Стоимость расходной части перевозки'].sum()/df['Стоимость расходной части перевозки'].sum()

df = df[df['Перевозка одним днем']==1]


# #что в фактическом расстоянии
# df['Фактическое расстояние'].hist(bins=500)
#
# #распредление ам по количеству маршрутов день
# df.groupby('Расчетный номер машины (из ТМС, 1С УАТ)')['Дата начала перевозки'].count().hist()
# #! одна и та же машина учавствует более одного раза, до 15


#распредление маршрутов и транспортных средств по УФПС, по дням. Какая фактическая размерность оптимизируемого вектора?!
# dpt = pd.pivot_table(df, index=['Дата начала перевозки', 'УФПС начала перевозки'], values='Наименование маршрута', aggfunc='count')
#! от 195 до 390 включая

#---------------------------------------------------------------------------------------------------
#vehicles = [
#    {'тип': 'Грузовик A', 'грузоподъемность': 1000, 'стоимость_на_1_км': 5, 'количество': 3},
#    {'тип': 'Грузовик B', 'грузоподъемность': 2000, 'стоимость_на_1_км': 7, 'количество': 2}


def prepare_data(df, ufps, data_transport):
    #условиие на формирование парка
    df = df[(df['УФПС начала перевозки'] == ufps) & (df['Дата начала перевозки'] == data_transport)]



    df_car = df[['Расчетный номер машины (из ТМС, 1С УАТ)', 'Наличие автомашины в собственном парке Почты', \
        'Фактическая грузоподъемность ТС (в кг)', 'УФПС начала перевозки', 'Модель ТС', 'затраты на км']]

    len(df_car) # 9342
    df_car = df_car.dropna()
    len(df_car) # 3401
    df_car.dtypes

    dpt_cars_M = pd.pivot_table(df_car[(df_car['Наличие автомашины в собственном парке Почты']==1) & \
                                       (df_car['УФПС начала перевозки']=='УФПС МОСКВЫ')], \
                                       index=['Модель ТС'], values='Расчетный номер машины (из ТМС, 1С УАТ)', aggfunc='nunique')



    dpt_cars_M_attrib = pd.pivot_table(df_car[(df_car['Наличие автомашины в собственном парке Почты']==1)], \
                                        index=['Модель ТС'], values=['Фактическая грузоподъемность ТС (в кг)', 'затраты на км'], aggfunc='median')

    dpt_cars_M.reset_index(inplace=True)
    dpt_cars_M_attrib.reset_index(inplace=True)

    table_cars = dpt_cars_M_attrib.merge(dpt_cars_M)
    #table_cars.to_pickle(path + 'table_cars_moscow.pkl')
    #---------------------------------------------------------------------------------------------------
    #deliveries = [
    #    {'точка': 'Точка 1', 'вес_груза': 1500, 'расстояние': 50},
    #    {'точка': 'Точка 2', 'вес_груза': 3000, 'расстояние': 70}
    #]

    #todo:добавляем стоимость перевозок (поскольку у нас агрегат из маршрутов)
    df_target = df[['Дата начала перевозки', 'УФПС начала перевозки', 'Наименование маршрута', 'Фактическое расстояние', \
                    'Сумма весов емкостей из Сортмастер, кг', 'Стоимость расходной части перевозки']]
    df_target['Дата начала перевозки'].unique()

    len(df_target) #9342
    df_target = df_target.dropna()
    len(df_target) #3401

    #для примера берем только Москву и 2024-02-05T00:00:00.000000000
    #df_target_M_d = df_target[(df_target['УФПС начала перевозки']=='УФПС МОСКВЫ') & \
    #(df_target['Дата начала перевозки']=='2024-02-05T00:00:00.000000000')] условие уже задано выше! для всего df


    #разброс фактических расстояний для одного маршрута в рамках одного дня
    dpt_target_M_d_distance = pd.pivot_table(df_target , index='Наименование маршрута', \
                                    values='Фактическое расстояние', aggfunc='mean') #aggfunc=['min','max']



    dpt_target_M_d_kg = pd.pivot_table(df_target , index='Наименование маршрута', \
                                    values=['Сумма весов емкостей из Сортмастер, кг', 'Стоимость расходной части перевозки'], aggfunc='sum')


    dpt_target_M_d_distance.reset_index(inplace=True)
    dpt_target_M_d_kg.reset_index(inplace=True)

    table_target = dpt_target_M_d_distance.merge(dpt_target_M_d_kg)

    #table_target.to_pickle(path + 'table_target_moscow_1day.pkl')
    #---------------------------------------------------------------------------------------------------

    df_target_distance =df[df['Наличие автомашины в собственном парке Почты']==1][['Наименование маршрута', 'Фактическое расстояние']]


    df_target_distance =df_target_distance.dropna()

    dpt_target_distance = pd.pivot_table(df_target_distance, index='Наименование маршрута', values='Фактическое расстояние', \
                                         aggfunc='mean').reset_index()

    len(dpt_target_distance) # 1197

    df_car_velocity = df[df['Наличие автомашины в собственном парке Почты']==1][['Модель ТС', 'Средняя скорость передвижения']]
    df_car_velocity = df_car_velocity.dropna()

    dpt_car_velocity = pd.pivot_table(df_car_velocity , index = 'Модель ТС', values='Средняя скорость передвижения', \
                                         aggfunc='mean').reset_index()


    dpt_car_velocity_real = dpt_cars_M_attrib.merge(dpt_car_velocity)
    len(dpt_car_velocity_real) #12


    df_combined = pd.merge(dpt_target_distance, dpt_car_velocity_real, how='cross')
    df_combined['время тс на маршруте'] = df_combined['Фактическое расстояние']/df_combined['Средняя скорость передвижения']

    #df_combined.to_pickle(path + 'table_cars_target_time_edit_m.pkl')

    return table_cars, table_target, df_combined

ufps = 'УФПС МОСКВЫ'
data_transport = '2024-02-05T00:00:00.000000000'


table_cars_moscow, table_target_moscow_1day, table_cars_target_time_edit_m = prepare_data(df, ufps, data_transport)


########-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
table_cars_target_time_edit_m = table_cars_target_time_edit_m[table_cars_target_time_edit_m['Наименование маршрута'].isin(table_target_moscow_1day['Наименование маршрута'])]

#смотрим, что со временем
#table_cars_target_time_edit_m['время тс на маршруте'].describe() #max 191...
table_cars_target_time_edit_m['время тс на маршруте'] = table_cars_target_time_edit_m['время тс на маршруте'].clip(upper=24)
#теперь максимум = 24

#марки машин
len(table_cars_moscow['Модель ТС'].unique()) #12
len(table_cars_target_time_edit_m['Модель ТС'].unique()) #12
#тут Ок!


### формирование словарей
vehicles = [{'тип': row['Модель ТС'],
             'Фактическая грузоподъемность ТС (в кг)': row['Фактическая грузоподъемность ТС (в кг)'],
             'затраты_на_км': row['затраты на км'],
             'Расчетный номер машины (из ТМС, 1С УАТ)': row['Расчетный номер машины (из ТМС, 1С УАТ)']} for _, row in table_cars_moscow.iterrows()]

deliveries = [{'точка': row['Наименование маршрута'],
               'Сумма весов емкостей из Сортмастер, кг': row['Сумма весов емкостей из Сортмастер, кг'],
               'Фактическое расстояние': row['Фактическое расстояние']} for _, row in table_target_moscow_1day.iterrows()]
times = {}
for _, row in table_cars_target_time_edit_m.iterrows():
    key = (row['Наименование маршрута'], row['Модель ТС'])
    value = {'время': row['время тс на маршруте']}
    times[key] = value



C:\Users\Alinvare\AppData\Local\Temp\ipykernel_7100


In [2]:

# Фильтрация данных
filtered_df = df[(df['УФПС начала перевозки'] == ufps) & (df['Дата начала перевозки'] == data_transport)]

# Вывод отфильтрованных данных
filtered_df


,"Расчетный номер машины (из ТМС, 1С УАТ)",Наличие автомашины в собственном парке Почты,"Сумма весов емкостей из Сортмастер, кг",Средняя скорость передвижения,Дата начала перевозки,Дата начала перевозки локальное время,Дата окончания перевозки (локальное время),Фактическая грузоподъемность ТС (в кг),УФПС начала перевозки,Наименование маршрута,Модель ТС,Фактическое расстояние,Стоимость расходной части перевозки,"Расчет затрат (руб) на (тонна*км) по перевозкам и из ТМС, и из 1с уат",затраты на км,Перевозка одним днем
159,Х 926 АК 799,1.0,935.315,13.33,2024-02-05,2024-02-05 07:00:00,2024-02-05 13:00:00,1000.0,УФПС МОСКВЫ,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,FORD Transit,80.0,5310.79,51.601147,66.384875,1
160,А 740 НР 750,1.0,172.428,5.33,2024-02-05,2024-02-05 09:00:00,2024-02-05 13:00:00,500.0,УФПС МОСКВЫ,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,LADA Largus,32.0,1751.77,134.855273,54.742812,1
162,К 791 ТУ 777,1.0,1588.463,4.50,2024-02-05,2024-02-05 07:00:00,2024-02-05 13:00:00,1500.0,УФПС МОСКВЫ,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,ГАЗ A31R32,27.0,2529.36,69.794702,93.680000,1
165,Е 787 КЕ 790,1.0,2172.214,14.63,2024-02-05,2024-02-05 07:00:00,2024-02-05 15:00:00,1000.0,УФПС МОСКВЫ,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,ГАЗ 27057,117.0,6091.09,51.967324,52.060598,1
296,В 646 СХ 777,1.0,1262.386,19.47,2024-02-05,2024-02-05 09:55:00,2024-02-05 21:45:00,1500.0,УФПС МОСКВЫ,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,ГАЗ A31R32,219.0,13108.90,45.234300,59.857991,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4568,С 384 ХХ 799,NaN,2128.769,NaN,2024-02-05,2024-02-05 11:25:00,2024-02-05 22:30:00,NaN,УФПС МОСКВЫ,ЛЦ ВНУКОВО-2 - ГОПС МОСКВА 81 - ГОПС МОСКВА 64...,NaN,NaN,15233.33,NaN,NaN,1
4573,Т 043 ТХ 69,NaN,1385.482,NaN,2024-02-05,2024-02-05 10:05:00,2024-02-05 20:10:00,NaN,УФПС МОСКВЫ,ЛЦ ВНУКОВО-2 - ГОПС МОСКВА 476 - ГОПС МОСКВА 4...,NaN,NaN,14233.33,NaN,NaN,1
4580,Р 596 ОО 777,1.0,0.000,NaN,2024-02-05,2024-02-05 08:00:00,2024-02-05 17:00:00,500.0,УФПС МОСКВЫ,ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) - ...,NaN,NaN,8362.45,NaN,NaN,1
4581,Р 907 СХ 799,1.0,0.000,NaN,2024-02-05,2024-02-05 08:00:00,2024-02-05 17:00:00,500.0,УФПС МОСКВЫ,ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) - ...,NaN,NaN,8362.45,NaN,NaN,1


In [3]:
df_cleaned = filtered_df.dropna().copy()

In [4]:
df_cleaned

,"Расчетный номер машины (из ТМС, 1С УАТ)",Наличие автомашины в собственном парке Почты,"Сумма весов емкостей из Сортмастер, кг",Средняя скорость передвижения,Дата начала перевозки,Дата начала перевозки локальное время,Дата окончания перевозки (локальное время),Фактическая грузоподъемность ТС (в кг),УФПС начала перевозки,Наименование маршрута,Модель ТС,Фактическое расстояние,Стоимость расходной части перевозки,"Расчет затрат (руб) на (тонна*км) по перевозкам и из ТМС, и из 1с уат",затраты на км,Перевозка одним днем
159,Х 926 АК 799,1.0,935.315,13.33,2024-02-05,2024-02-05 07:00:00,2024-02-05 13:00:00,1000.0,УФПС МОСКВЫ,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,FORD Transit,80.0,5310.79,51.601147,66.384875,1
160,А 740 НР 750,1.0,172.428,5.33,2024-02-05,2024-02-05 09:00:00,2024-02-05 13:00:00,500.0,УФПС МОСКВЫ,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,LADA Largus,32.0,1751.77,134.855273,54.742812,1
162,К 791 ТУ 777,1.0,1588.463,4.50,2024-02-05,2024-02-05 07:00:00,2024-02-05 13:00:00,1500.0,УФПС МОСКВЫ,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,ГАЗ A31R32,27.0,2529.36,69.794702,93.680000,1
165,Е 787 КЕ 790,1.0,2172.214,14.63,2024-02-05,2024-02-05 07:00:00,2024-02-05 15:00:00,1000.0,УФПС МОСКВЫ,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,ГАЗ 27057,117.0,6091.09,51.967324,52.060598,1
296,В 646 СХ 777,1.0,1262.386,19.47,2024-02-05,2024-02-05 09:55:00,2024-02-05 21:45:00,1500.0,УФПС МОСКВЫ,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,ГАЗ A31R32,219.0,13108.90,45.234300,59.857991,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,Н 021 АН 799,1.0,62.572,16.90,2024-02-05,2024-02-05 08:57:00,2024-02-05 19:00:12,1000.0,УФПС МОСКВЫ,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,FORD Transit,169.0,8423.69,107.664749,49.844320,1
575,Р 366 ТО 799,1.0,1138.133,20.67,2024-02-05,2024-02-05 08:45:00,2024-02-05 20:45:00,1000.0,УФПС МОСКВЫ,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,FORD Transit,248.0,14064.38,66.870067,56.711210,1
1385,Р 907 СХ 799,1.0,0.000,11.42,2024-02-05,2024-02-05 19:00:24,2024-02-05 23:31:24,500.0,УФПС МОСКВЫ,ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) - ...,LADA Largus,137.0,4399.53,87.990600,32.113358,1
1387,Х 099 ТО 777,1.0,0.000,14.83,2024-02-05,2024-02-05 19:00:38,2024-02-05 23:41:38,500.0,УФПС МОСКВЫ,ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) - ...,LADA Largus,178.0,7726.44,154.528800,43.406966,1


In [4]:
#df_cleaned = df.dropna().copy()

# Преобразуем столбцы 'Дата начала перевозки локальное время' и 'Дата окончания перевозки локальное время' в формат datetime
df_cleaned['Дата начала перевозки локальное время'] = pd.to_datetime(df_cleaned['Дата начала перевозки локальное время'])
df_cleaned['Дата окончания перевозки (локальное время)'] = pd.to_datetime(df_cleaned['Дата окончания перевозки (локальное время)'])

# Вычисляем время маршрута как разницу между 'Дата окончания перевозки (локальное время)' и 'Дата начала перевозки локальное время'
df_cleaned['Время маршрута'] = (df_cleaned['Дата окончания перевозки (локальное время)'] - df_cleaned['Дата начала перевозки локальное время']).dt.total_seconds() / 3600  # переводим в часы
df_cleaned=df_cleaned.reset_index()

In [19]:
time_by_vehicle_type = {}
utilization_by_vehicle_type = {}

# Проходим по каждой строке датафрейма
for index, row in df_cleaned.iterrows():
    vehicle_type = row['Модель ТС']  # Получаем тип машины
    time_ = row['Время маршрута']  # Получаем время маршрута
    # Если тип машины уже есть в словаре, добавляем время к существующей сумме
    if vehicle_type in time_by_vehicle_type:
        time_by_vehicle_type[vehicle_type] += time_
    else:
        time_by_vehicle_type[vehicle_type] = time_  # Иначе создаем новую запись

count_by_model = df_cleaned['Модель ТС'].value_counts()
count_by_model_df = pd.DataFrame({'Модель ТС': count_by_model.index, 'Количество маршрутов': count_by_model.values})
# Рассчитываем процент утилизации для каждой модели машины
for vehicle_type, total_time in time_by_vehicle_type.items():
    total_possible_time = 24 * count_by_model_df[count_by_model_df['Модель ТС'] == vehicle_type]['Количество маршрутов'].sum()# Предположим, что каждая машина может быть использована в течение 24 часов
    utilization_percentage = (total_time / total_possible_time) * 100
    utilization_by_vehicle_type[vehicle_type] = utilization_percentage

# Рассчитываем количество маршрутов для каждой модели машины


# Создаем новый датафрейм на основе полученных данных
summary_df = pd.DataFrame({
    'Модель ТС': utilization_by_vehicle_type.keys(),
    'Процент утилизации': utilization_by_vehicle_type.values(),
    'Количество маршрутов': count_by_model.values,
    'Дата начала перевозки': df_cleaned['Дата начала перевозки'][0]
})

# Выводим результаты
summary_df

FORD Transit
LADA Largus
ГАЗ A31R32
ГАЗ 27057
IVECO Daily
ГАЗ С41R92
HINO 300
КАМАЗ 5490
HINO 500
IVECO Stralis
ГАЗ C41R33
КАМАЗ 5325


,Модель ТС,Процент утилизации,Количество маршрутов,Дата начала перевозки
0,FORD Transit,43.161962,46,2024-02-05
1,LADA Largus,37.290088,44,2024-02-05
2,ГАЗ A31R32,44.493981,10,2024-02-05
3,ГАЗ 27057,33.333333,6,2024-02-05
4,IVECO Daily,30.208333,5,2024-02-05
5,ГАЗ С41R92,39.930556,4,2024-02-05
6,HINO 300,38.194444,2,2024-02-05
7,КАМАЗ 5490,46.527778,1,2024-02-05
8,HINO 500,48.750000,1,2024-02-05
9,IVECO Stralis,53.703704,1,2024-02-05


In [7]:
# Заполняем список данными
for car_type, total_time in total_time_by_car_type.items():
    total_possible_time = 24 * table_cars_moscow_results[table_cars_moscow_results['Модель ТС'] == car_type]['Расчетный номер машины (из ТМС, 1С УАТ)'].sum()
    utilization_percentage = (total_time / total_possible_time) * 100
    used_cars_count = table_cars_moscow_results[table_cars_moscow_results['Модель ТС'] == car_type]['Количество рейсов'].sum()
    
    # Добавляем данные в список
    utilization_data.append({
        'Модель ТС': car_type,
        'Количество используемых машин (Оптимизация)': used_cars_count,
        'Процент утилизации(Оптимизация)': utilization_percentage
    })


12

In [8]:
count_by_model = df['Модель ТС'].value_counts()

# Выводим результаты
len(count_by_model)

23

In [6]:
import time

def optimize_delivery_integer(vehicles, deliveries, times):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    num_deliveries = len(deliveries)
    num_vehicles = len(vehicles)

    x = np.array([[solver.IntVar(0, solver.infinity(), f'x[{i},{j}]') for j in range(num_vehicles)] for i in range(num_deliveries)])

    # Create the cost matrix using the provided data
    cost_matrix = np.array([[vehicles[j]['затраты_на_км'] * delivery['Фактическое расстояние'] for j in range(num_vehicles)] for delivery in deliveries])

    solver.Minimize(solver.Sum(x[i, j] * cost_matrix[i, j] for i in range(num_deliveries) for j in range(num_vehicles)))

    for i in range(num_deliveries):
        solver.Add(solver.Sum(x[i, j] * vehicles[j]['Фактическая грузоподъемность ТС (в кг)'] for j in range(num_vehicles)) >= deliveries[i]['Сумма весов емкостей из Сортмастер, кг'])

    for j in range(num_vehicles):
        for i in range(num_deliveries):              #ставится коментарий тут для отключения считания с временем
            # Access the time for this vehicle and delivery point pair from the 'times' table
            time_value = times[(deliveries[i]['точка'], vehicles[j]['тип'])]['время']                      #ставится коментарий тут для отключения считания с временем
            solver.Add(x[i, j] * time_value <= 24)               #ставится коментарий тут для отключения считания с временем
            #print (time_value)

        solver.Add(solver.Sum(x[i, j] for i in range(num_deliveries)) <= vehicles[j]['Расчетный номер машины (из ТМС, 1С УАТ)'])

    start_time = time.time()  # Засекаем время начала оптимизации
    status = solver.Solve()
    execution_time = time.time() - start_time  # Вычисляем время выполнения оптимизации

    result = {
        'status': status,
        'objective_value': solver.Objective().Value(),
        'x_values': {(i, j): x[i, j].solution_value() for j in range(num_vehicles) for i in range(num_deliveries)},
        'execution_time': execution_time  # Добавляем время выполнения в результат
    }

    return result


table_target_moscow_1day_results=table_target_moscow_1day.copy()
table_target_moscow_1day_results['Количество рейсов']=0
table_target_moscow_1day_results['Модель ТС']=[None] * len(table_target_moscow_1day_results)
table_target_moscow_1day_results['Стоимость']=0

table_cars_moscow_results=table_cars_moscow.copy()
table_cars_moscow_results['Количество рейсов']=0
table_cars_moscow_results['Наименование маршрута']=[None] * len(table_cars_moscow_results)

result_integer = optimize_delivery_integer(vehicles, deliveries, times)
print("Результаты оптимизации:")
print(f"Статус оптимизации: {result_integer['status']}")
print(f"Общая стоимость доставки: {result_integer['objective_value']}")
#summing_testing=0
for (i, j), value in result_integer['x_values'].items():
    if value>0:
        #print(f"Грузовик {vehicles[j]['тип']} отправляется на {deliveries[i]['точка']}, количество рейсов: {value}")
        car_index=table_cars_moscow[(table_cars_moscow['Модель ТС']== vehicles[j]['тип'] )].index[0] 
        delivers_index=table_target_moscow_1day[(table_target_moscow_1day['Наименование маршрута']==deliveries[i]['точка'])].index[0] 
        #summing_testing+=table_cars_moscow.loc[car_index,'затраты на км']*table_target_moscow_1day.loc[delivers_index,'Фактическое расстояние']*value

        table_cars_moscow_results.loc[car_index, 'Количество рейсов'] += value
        if isinstance(table_cars_moscow_results.loc[car_index, 'Наименование маршрута'], list):
            table_cars_moscow_results.loc[car_index, 'Наименование маршрута'].append(deliveries[i]['точка'])
        else:
            table_cars_moscow_results.loc[car_index, 'Наименование маршрута'] = [deliveries[i]['точка']]

        target_index = table_target_moscow_1day_results[
            table_target_moscow_1day_results['Наименование маршрута'] == deliveries[i]['точка']].index[0]

        table_target_moscow_1day_results.loc[target_index, 'Количество рейсов'] += value
        # Получаем текущее значение столбца 'Модель ТС'
        current_value = table_target_moscow_1day_results.loc[target_index, 'Модель ТС']

        # Проверяем, является ли текущее значение списком
        if isinstance(current_value, list):
            # Если значение уже список, добавляем элемент
            current_value.append(vehicles[j]['тип'])
        else:
            # Если значение не список (NoneType), создаем новый список с одним элементом
            table_target_moscow_1day_results.loc[target_index, 'Модель ТС'] = [vehicles[j]['тип']]
        
        table_target_moscow_1day_results.loc[target_index, 'Стоимость'] +=table_cars_moscow.loc[car_index,'затраты на км']*table_target_moscow_1day.loc[delivers_index,'Фактическое расстояние']*value
print(f"Время выполнения: {result_integer['execution_time']} секунд")

########-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


table_target_moscow_1day_results.columns

table_target_moscow_1day_results['Стоимость расходной части перевозки'].sum()/table_target_moscow_1day_results['Стоимость'].sum()

print (sum(table_target_moscow_1day_results['Стоимость']))

Результаты оптимизации:
Статус оптимизации: 0
Общая стоимость доставки: 953707.2639683725
Время выполнения: 0.04082751274108887 секунд
953707.2639683724


In [7]:
table_target_moscow_1day_results

,Наименование маршрута,Фактическое расстояние,Стоимость расходной части перевозки,"Сумма весов емкостей из Сортмастер, кг",Количество рейсов,Модель ТС,Стоимость
0,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,117.0,6091.09,2172.214,1,[IVECO Stralis],12700.997485
1,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,27.0,2529.36,1588.463,2,[FORD Transit],4985.284075
2,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,80.0,5310.79,935.315,1,[FORD Transit],7385.606037
3,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,32.0,1751.77,172.428,1,[LADA Largus],2403.953119
4,ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) - ...,58.0,7430.27,0.000,0,None,0.000000
...,...,...,...,...,...,...,...
85,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,71.0,10443.14,1262.030,2,"[FORD Transit, LADA Largus]",11888.496341
86,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,70.0,16724.90,0.000,0,None,0.000000
87,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,115.0,10096.47,633.331,1,[FORD Transit],10616.808679
88,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,128.0,10477.24,1110.421,1,[ГАЗ A31R32],12183.980000


In [14]:
table_cars_moscow_results

,Модель ТС,Фактическая грузоподъемность ТС (в кг),затраты на км,"Расчетный номер машины (из ТМС, 1С УАТ)",Количество рейсов,Наименование маршрута
0,FORD Transit,1000.0,92.320075,46,28,[АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП...
1,HINO 300,4000.0,84.185373,1,1,[ТРАНСПОРТНАЯ ГРУППА № 1 АГАПОУИМП (АТП МОСКВЫ...
2,HINO 500,10500.0,71.030611,5,5,[ТРАНСПОРТНАЯ ГРУППА № 1 АГАПОУИМП (АТП МОСКВЫ...
3,IVECO Daily,1000.0,113.050156,1,0,None
4,IVECO Stralis,10000.0,108.555534,6,6,[АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП...
5,LADA Largus,500.0,75.123535,44,28,[АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП...
6,ГАЗ 27057,1000.0,52.060598,1,1,[ТРАНСПОРТНАЯ ГРУППА № 1 АГАПОУИМП (АТП МОСКВЫ...
7,ГАЗ A31R32,1500.0,95.187344,10,10,[ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) -...
8,ГАЗ C41R33,4000.0,73.748669,4,4,[ТРАНСПОРТНАЯ ГРУППА № 1 АГАПОУИМП (АТП МОСКВЫ...
9,ГАЗ С41R92,2500.0,111.483182,1,1,[ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) -...


In [8]:
# Сначала создадим словарь для хранения суммарного времени маршрутов по типам машин
total_time_by_car_type = {}

# Перебираем результаты оптимизации
for (i, j), value in result_integer['x_values'].items():
    if value > 0:
        car_type = vehicles[j]['тип']  # Получаем тип машины
        time_value = times[(deliveries[i]['точка'], car_type)]['время']  # Получаем время маршрута

        # Если тип машины уже есть в словаре, добавляем время маршрута
        if car_type in total_time_by_car_type:
            total_time_by_car_type[car_type] += time_value * value
        else:  # Иначе, создаем новую запись
            total_time_by_car_type[car_type] = time_value * value



In [20]:
import pandas as pd

# Создаем список для хранения данных
utilization_data = []

# Заполняем список данными
for car_type, total_time in total_time_by_car_type.items():
    total_possible_time = 24 * table_cars_moscow_results[table_cars_moscow_results['Модель ТС'] == car_type]['Расчетный номер машины (из ТМС, 1С УАТ)'].sum()
    utilization_percentage = (total_time / total_possible_time) * 100
    used_cars_count = table_cars_moscow_results[table_cars_moscow_results['Модель ТС'] == car_type]['Количество рейсов'].sum()
    
    # Добавляем данные в список
    utilization_data.append({
        'Модель ТС': car_type,
        'Количество используемых машин (Оптимизация)': used_cars_count,
        'Процент утилизации(Оптимизация)': utilization_percentage
    })

# Создаем DataFrame из списка данных
utilization_df = pd.DataFrame(utilization_data)

# Выводим полученный DataFrame
utilization_df


,Модель ТС,Количество используемых машин (Оптимизация),Процент утилизации(Оптимизация)
0,FORD Transit,28,21.137812
1,HINO 300,1,41.894665
2,HINO 500,5,84.637357
3,IVECO Stralis,6,30.426730
4,LADA Largus,28,26.954054
5,ГАЗ 27057,1,65.219868
6,ГАЗ A31R32,10,46.435257
7,ГАЗ C41R33,4,56.737297
8,ГАЗ С41R92,1,43.572985
9,КАМАЗ 5325,2,34.626569


In [21]:
summary_df

,Модель ТС,Процент утилизации,Количество маршрутов,Дата начала перевозки
0,FORD Transit,43.161962,46,2024-02-05
1,LADA Largus,37.290088,44,2024-02-05
2,ГАЗ A31R32,44.493981,10,2024-02-05
3,ГАЗ 27057,33.333333,6,2024-02-05
4,IVECO Daily,30.208333,5,2024-02-05
5,ГАЗ С41R92,39.930556,4,2024-02-05
6,HINO 300,38.194444,2,2024-02-05
7,КАМАЗ 5490,46.527778,1,2024-02-05
8,HINO 500,48.750000,1,2024-02-05
9,IVECO Stralis,53.703704,1,2024-02-05


In [32]:
table_for_vis= pd.DataFrame()
table_for_vis = table_target_moscow_1day_results.explode('Модель ТС')

In [34]:
table_for_vis=table_for_vis.drop(['Количество рейсов','Стоимость'],axis=1)

In [36]:
used_cars_count = table_cars_moscow_results.groupby('Модель ТС')['Количество рейсов'].sum()
print("Количество используемых машин разного типа:")
print(used_cars_count)

Количество используемых машин разного типа:
Модель ТС
FORD Transit     28
HINO 300          1
HINO 500          5
IVECO Daily       0
IVECO Stralis     6
LADA Largus      28
ГАЗ 27057         1
ГАЗ A31R32       10
ГАЗ C41R33        4
ГАЗ С41R92        1
КАМАЗ 5325        2
КАМАЗ 5490        1
Name: Количество рейсов, dtype: int64


In [37]:
used_cars_count

Модель ТС
FORD Transit     28
HINO 300          1
HINO 500          5
IVECO Daily       0
IVECO Stralis     6
LADA Largus      28
ГАЗ 27057         1
ГАЗ A31R32       10
ГАЗ C41R33        4
ГАЗ С41R92        1
КАМАЗ 5325        2
КАМАЗ 5490        1
Name: Количество рейсов, dtype: int64

In [35]:
table_for_vis

,Наименование маршрута,Фактическое расстояние,Стоимость расходной части перевозки,"Сумма весов емкостей из Сортмастер, кг",Модель ТС
0,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,117.0,6091.09,2172.214,IVECO Stralis
1,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,27.0,2529.36,1588.463,FORD Transit
2,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,80.0,5310.79,935.315,FORD Transit
3,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,32.0,1751.77,172.428,LADA Largus
4,ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) - ...,58.0,7430.27,0.000,None
...,...,...,...,...,...
85,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,71.0,10443.14,1262.030,LADA Largus
86,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,70.0,16724.90,0.000,None
87,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,115.0,10096.47,633.331,FORD Transit
88,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,128.0,10477.24,1110.421,ГАЗ A31R32


In [30]:
table_target_moscow_1day_results

,Наименование маршрута,Фактическое расстояние,Стоимость расходной части перевозки,"Сумма весов емкостей из Сортмастер, кг",Количество рейсов,Модель ТС,Стоимость
0,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,117.0,6091.09,2172.214,1,[IVECO Stralis],12700.997485
1,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,27.0,2529.36,1588.463,2,[FORD Transit],4985.284075
2,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,80.0,5310.79,935.315,1,[FORD Transit],7385.606037
3,АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП ...,32.0,1751.77,172.428,1,[LADA Largus],2403.953119
4,ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) - ...,58.0,7430.27,0.000,0,None,0.000000
...,...,...,...,...,...,...,...
85,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,71.0,10443.14,1262.030,2,"[FORD Transit, LADA Largus]",11888.496341
86,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,70.0,16724.90,0.000,0,None,0.000000
87,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,115.0,10096.47,633.331,1,[FORD Transit],10616.808679
88,ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) - ...,128.0,10477.24,1110.421,1,[ГАЗ A31R32],12183.980000


In [31]:
table_cars_moscow_results

,Модель ТС,Фактическая грузоподъемность ТС (в кг),затраты на км,"Расчетный номер машины (из ТМС, 1С УАТ)",Количество рейсов,Наименование маршрута
0,FORD Transit,1000.0,92.320075,46,28,[АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП...
1,HINO 300,4000.0,84.185373,1,1,[ТРАНСПОРТНАЯ ГРУППА № 1 АГАПОУИМП (АТП МОСКВЫ...
2,HINO 500,10500.0,71.030611,5,5,[ТРАНСПОРТНАЯ ГРУППА № 1 АГАПОУИМП (АТП МОСКВЫ...
3,IVECO Daily,1000.0,113.050156,1,0,None
4,IVECO Stralis,10000.0,108.555534,6,6,[АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП...
5,LADA Largus,500.0,75.123535,44,28,[АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП...
6,ГАЗ 27057,1000.0,52.060598,1,1,[ТРАНСПОРТНАЯ ГРУППА № 1 АГАПОУИМП (АТП МОСКВЫ...
7,ГАЗ A31R32,1500.0,95.187344,10,10,[ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ) -...
8,ГАЗ C41R33,4000.0,73.748669,4,4,[ТРАНСПОРТНАЯ ГРУППА № 1 АГАПОУИМП (АТП МОСКВЫ...
9,ГАЗ С41R92,2500.0,111.483182,1,1,[ТРАНСПОРТНАЯ ГРУППА № 3 ААППОМ (АТП МОСКВЫ) -...


In [20]:
import pandas as pd

# Создаем исходную таблицу
data = {
    'Наименование маршрута': [
        'АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП',
        'ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ)',
        'ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ)',
        # Добавьте остальные данные
    ],
    'Модель ТС': [
        ['IVECO Stralis', 'FORD'],
        ['FORD Transit'],
        [None]
        # Добавьте остальные данные
    ],
    'Cost':[
        '12',
        '15',
        '0'    ]
}
df = pd.DataFrame(data)

# Разделяем массивы моделей на отдельные строки
df = df.explode('Модель ТС')

# Сбрасываем индексы для удобства
df.reset_index(drop=True, inplace=True)

# Выводим преобразованную таблицу
print(df)


                           Наименование маршрута      Модель ТС Cost
0  АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП  IVECO Stralis   12
1  АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП           FORD   12
2    ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ)   FORD Transit   15
3    ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ)           None    0


In [13]:
# Преобразование массива моделей в одну модель (первую из массива)
df['Модель ТС'] = df['Модель ТС'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

# Выводим преобразованную таблицу
print(df)

                           Наименование маршрута      Модель ТС
0  АТП ДОМОДЕДОВСКОГО ПОЧТАМТА - ДОМОДЕДОВО УООП  IVECO Stralis
1    ТРАНСПОРТНАЯ ГРУППА № 1 ААППОМ (АТП МОСКВЫ)   FORD Transit
